In [1]:
# TEST: Imports
from seccom import Seccom
import datetime, secrets, binascii, os
from cryptography import x509
from cryptography.x509.oid import NameOID
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import serialization, hashes
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.primitives.asymmetric import ec, rsa
from cryptography.hazmat.primitives.kdf.concatkdf import ConcatKDFHMAC
from Cryptodome.Cipher import PKCS1_OAEP

In [2]:
# REQ: Peer Keys
# B Keys
ec_private_key_b = ec.generate_private_key(ec.SECP256R1(), default_backend())
ec_public_key_b = ec_private_key_b.public_key()
ec_public_key_b_hex = ec_public_key_b.public_bytes(encoding=serialization.Encoding.PEM,
    format=serialization.PublicFormat.SubjectPublicKeyInfo
).hex()
# Generate a new RSA private key
rsa_private_key_b = rsa.generate_private_key(
    public_exponent=65537,
    key_size=2048,
    backend=default_backend()
)
# Define the subject of the certificate
subject = x509.Name([
    x509.NameAttribute(NameOID.COUNTRY_NAME, "DE"),
    x509.NameAttribute(NameOID.STATE_OR_PROVINCE_NAME, ""),
    x509.NameAttribute(NameOID.LOCALITY_NAME, ""),
    x509.NameAttribute(NameOID.ORGANIZATION_NAME, ""),
    x509.NameAttribute(NameOID.COMMON_NAME, "0.0.0.0"),
])
# Create the certificate
cert_b = (
    x509.CertificateBuilder()
    .subject_name(subject)
    .issuer_name(subject)
    .public_key(rsa_private_key_b.public_key())
    .serial_number(x509.random_serial_number())
    .not_valid_before(datetime.datetime.utcnow())
    .not_valid_after(datetime.datetime.utcnow() + datetime.timedelta(days=365))
    .add_extension(
        x509.BasicConstraints(ca=False, path_length=None), critical=True,
    )
    .add_extension(
        x509.SubjectAlternativeName([x509.DNSName("127.0.0.1")]), critical=False,
    )
    .sign(rsa_private_key_b, hashes.SHA256(), default_backend())
)
# Print the private key and certificate in PEM format
private_key_pem_b = rsa_private_key_b.private_bytes(
    encoding=serialization.Encoding.PEM,
    format=serialization.PrivateFormat.TraditionalOpenSSL,
    encryption_algorithm=serialization.NoEncryption()
)
cert_pem_b = cert_b.public_bytes(encoding=serialization.Encoding.PEM)

In [3]:
# TEST: Initialization
seccom = Seccom()

In [4]:
# TEST: ECDHE
resultEcdhe = seccom.ecdh(ec_public_key_b_hex)

In [5]:
# TEST: RSA Exchange
iv = os.urandom(16)
cipher = Cipher(algorithms.AES(bytes.fromhex(resultEcdhe['sharedSecret'])),
                modes.CBC(iv), backend=default_backend())
encryptor = cipher.encryptor()
encPKRSA = encryptor.update(cert_pem_b)
resultRsaE = seccom.rsa(encPKRSA.hex(), iv.hex(), resultEcdhe['sharedSecret'])

In [6]:
# TEST: Encrypt and Decrypt
inputPlaintext="abc123!)?"
resultCipherText = seccom.encrypt(inputPlaintext.encode('utf-8'), cert_pem_b, resultEcdhe['sharedSecret'])
seccom.rsa_private_key = rsa_private_key_b
resultDec_plaintext = seccom.decrypt(resultCipherText['ciphertext'], resultCipherText['iv'], resultEcdhe['sharedSecret'])

In [7]:
# RESULTS:
print('Output ECDHE Handling:')
print(resultEcdhe)
print('\nOutput RSA Handling:')
print(resultRsaE)
print('\nPlaintext Pre-Ciphering:')
print(inputPlaintext)
print('\nCiphertext:')
print(resultCipherText)
print('\nDecrypted Plaintext:')
print(resultDec_plaintext.decode('utf-8'))

Output ECDHE Handling:
{'sharedSecret': 'e387406ae618a6da6bcbeb04394ce7243393e372119350f4e121c6ca230e3135', 'payload': {'digest': 'a8f020bfe5787398b6d383a50071fd2dd9d5733fa03974c074b61eb5d57c4d6c', 'key': '2d2d2d2d2d424547494e205055424c4943204b45592d2d2d2d2d0a4d466b77457759484b6f5a497a6a3043415159494b6f5a497a6a304441516344516741456c4d4879506757434e72424b793963364e4433714b55706772754f2b0a514d3275637049555a57614c574b6d664e4e494e707247486b356167614f6751586234785a2f33446173476e4c786141766e616f573441594f773d3d0a2d2d2d2d2d454e44205055424c4943204b45592d2d2d2d2d0a'}}

Output RSA Handling:
{'payload': {'iv': 'b6a2fe7d2fb28b4ccc1ca08313d0af9a', 'ciphertext': '1a826367756fca1ab30b71b5f6ddb42a958615a4204edcfe8a8eaddd73b235344646cb09dd3bca590d4e47d2659667e4c22db80437098011182ad6eeb2f01640b17488bc6216ab83f856e0926702eced6d67b39dc154bb6c5b3842afe88e96f110bb04171b972f09e7a101c1250d06e706c1cd6bd0924504c01de29e8ff22ec59f2528204c2f9ae41ad3fe0d435a9dc04f3fbe4081ab575fcf3867491e140a4bff94fbcf1a6d43c81855d8

In [8]:
# REQ: Both 'Parties' for Handshake
sec1 = Seccom()
sec2 = Seccom()

In [9]:
# TEST: Initiate Handshake on Peer (ECDHE)
req11 = sec1.ec_public_key.public_bytes(
    encoding=serialization.Encoding.PEM,
    format=serialization.PublicFormat.SubjectPublicKeyInfo
).hex()
req12 = sec2.ec_public_key.public_bytes(
    encoding=serialization.Encoding.PEM,
    format=serialization.PublicFormat.SubjectPublicKeyInfo
).hex()

resp11 = sec2.ecdh(req11)
resp12 = sec1.ecdh(req12)

In [10]:
# TEST: Finalize Handshake (RSAE)
req21 = {}
req21['iv'] = os.urandom(16)
cipher = Cipher(algorithms.AES(bytes.fromhex(resp11['sharedSecret'])),
                modes.CBC(req21['iv']), backend=default_backend())
encryptor = cipher.encryptor()
req21['encPKRSA'] = encryptor.update(sec1.cert_pem)
req22 = {}
req22['iv'] = os.urandom(16)
cipher = Cipher(algorithms.AES(bytes.fromhex(resp11['sharedSecret'])),
                modes.CBC(req22['iv']), backend=default_backend())
encryptor = cipher.encryptor()
req22['encPKRSB'] = encryptor.update(sec2.cert_pem)

resp21 = sec2.rsa(req21['encPKRSA'].hex(), req21['iv'].hex(), resp12['sharedSecret'])
resp22 = sec1.rsa(req22['encPKRSB'].hex(), req22['iv'].hex(), resp11['sharedSecret'])

res21 = sec1.decrypt(resp21['payload']['ciphertext'], resp21['payload']['iv'], resp11['sharedSecret'])
res22 = sec2.decrypt(resp22['payload']['ciphertext'], resp22['payload']['iv'], resp11['sharedSecret'])

In [11]:
# RESULT: Handshake
print('Clear-text Key 1:')
print(sec1.cert.public_bytes(encoding=serialization.Encoding.PEM))
print('Clear-text Key 2:')
print(sec2.cert.public_bytes(encoding=serialization.Encoding.PEM))
print('Request Initial Key 1:')
print(req11)
print('Request Initial Key 2:')
print(req12)
print('Peer Response for First Request 1:')
print(resp11)
print('Peer Response for First Request 2:')
print(resp12)
print('Rsa Request 1:')
print(req21)
print('Rsa Request 2:')
print(req22)
print('Rsa Responese 1:')
print(resp21)
print('Rsa Responese 2:')
print(resp22)
print('Rsa Responese Deciphered 1:')
print(res21)
print('Rsa Responese Deciphered 2:')
print(res22)
print('Val 1')
print(str(sec1.cert.public_bytes(encoding=serialization.Encoding.PEM) == res22))
print('Val 2')
print(str(sec2.cert.public_bytes(encoding=serialization.Encoding.PEM) == res21))

Clear-text Key 1:
b'-----BEGIN CERTIFICATE-----\nMIIDNDCCAhygAwIBAgIUXfVtSQ4ZFnXoBaSuD6otHrTm6UUwDQYJKoZIhvcNAQEL\nBQAwQDELMAkGA1UEBhMCREUxCTAHBgNVBAgMADEJMAcGA1UEBwwAMQkwBwYDVQQK\nDAAxEDAOBgNVBAMMBzAuMC4wLjAwHhcNMjMwNTA4MTI1OTE4WhcNMjQwNTA3MTI1\nOTE4WjBAMQswCQYDVQQGEwJERTEJMAcGA1UECAwAMQkwBwYDVQQHDAAxCTAHBgNV\nBAoMADEQMA4GA1UEAwwHMC4wLjAuMDCCASIwDQYJKoZIhvcNAQEBBQADggEPADCC\nAQoCggEBANWxypQCUXNAMd5MgombGGs3AKemGxJwqcQpSrN3Wo6w9WG3AFyckrVx\nu9UNGNjI3LtpONBPwMqv7iC0p6p8YT/Z65RkRmobUA3ghMri+2/NDNLQA9dvrCID\n0ym9riz/XBKnBg5p5s7Cp5M3RUr+vau2dkhzgr6uS7vYgVYVi8nCr5R7E7WvdEmg\nyVDGBLeKA1LBP0vXXQ5+PU4oAR7z8GdlSAL8/sueoIX/wbcAq4psOQVPaawZwmPp\nkoaTYm9i/cma/sgM65kYtUKmihdQXtTUXI1tSMhVEXO0+eKJyZZzZBOJ5taT3BII\nDoMf95hPsAZ/gZxGzvnFoRNubztgNZsCAwEAAaMmMCQwDAYDVR0TAQH/BAIwADAU\nBgNVHREEDTALggkxMjcuMC4wLjEwDQYJKoZIhvcNAQELBQADggEBAMXJi0hPYbk8\nrAqsMp/eOQD6N0/S4jDq/fLkbTnEGHxgkr8TUujBb5HHUf2IpnTJB4aOF9CPsRhp\nJc5rRm0pFsJW3a+UHyXKXooQ32qKseM9BjrSgOR37PoVw7Y70jKTEAZ/fBcOSzdZ\nZjTLqToEHW9amKDBK3Ox+7j4ww7

In [12]:
#IMPORTS, CONSTANTS, TEST-FUNCTION:
import requests, json, string, random

apiTokenValue = 'LlLvXURqNHrp75iyz9Riwen4DJ2QcfDK78sCw4tgjgPF4gsFcIJuPbQfrUMm'
userTokenValue = 'hmLHka03PSxsE73bKnD4xNXlPmy6UF7oiVNHANEvxAZOCsXz0J9DX82JjyBR'
creatorTokenValue = 'q98lD6P6s4jQWO8yMUZalvQZqZ2pQq3QHZLnwQKhXPhocrVYIuWTEYBOJta9'

def testRequest(methode: str, url: str, headers: dict, cookies: dict or None, body: dict) -> dict or Exception:
    resp = {}
    if methode == 'POST':
        try:
            response = requests.post(url, headers=headers, cookies=cookies, data=json.dumps(body), verify=False)
            if response.status_code == 200:
                resp['payload']: dict = json.loads(response.text)
                resp['csrfToken'] = response.cookies['X-CSRFToken']
                resp['sessionId'] = response.cookies['sessionId']
            else:
                raise Exception('Request failed with status code ' + str(response.status_code))
        except:
            raise Exception('Request failed without status code ')
    elif methode == 'GET':
        try:
            response = requests.get(url, headers=headers, cookies=cookies, data=json.dumps(body), verify=False)
            if response.status_code == 200:
                resp['payload']: dict = json.loads(response.text)
                resp['csrfToken'] = response.cookies['X-CSRFToken']
                resp['sessionId'] = response.cookies['sessionId']
            else:
                raise Exception('Request failed with status code ' + str(response.status_code))
        except:
            raise Exception('Request failed without status code ')

    return resp

def createId(length: int) -> str:
    letters     = string.ascii_lowercase
    result_str  = ''.join(random.choice(letters) for i in range(length))
    return result_str

In [13]:
# TEST: Ecdhe Request
sec3 = Seccom()
#sec4 = Seccom()

ecPkHex = sec3.ec_public_key.public_bytes(
    encoding=serialization.Encoding.PEM,
    format=serialization.PublicFormat.SubjectPublicKeyInfo
).hex()

resp0 = {}
try:
    headers = {'Content-Type': 'application/json'}
    body = {'apiToken': apiTokenValue, 'ecdhPublic': ecPkHex}
    resp0 = testRequest('POST', 'https://0.0.0.0:22432/handshake', headers, None, body)
except:
    raise Exception('Request failed with/without status code ')

localEcdhe = sec3.ecdh(str(resp0['payload']['key']))
sharedSecret = localEcdhe['sharedSecret']
valEcdhe = localEcdhe['payload']['digest'] == resp0['payload']['digest']

/usr/lib/python3/dist-packages/urllib3/connectionpool.py:1015: InsecureRequestWarning: Unverified HTTPS request is being made to host '0.0.0.0'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


In [14]:
# TEST: Rsae Request
iv = os.urandom(16)
cipher = Cipher(algorithms.AES(bytes.fromhex(sharedSecret)),
                modes.CBC(iv), backend=default_backend())
encryptor = cipher.encryptor()
encSymRsaPk = encryptor.update(sec3.cert_pem)

resp1 = {}
try:
    cookies = {'sessionId': resp0['sessionId'], 'X-CSRFToken': resp0['csrfToken']}
    headers = {'Content-Type': 'application/json'}
    body = {'apiToken': apiTokenValue, 'iv': iv.hex(), 'key': encSymRsaPk.hex()}
    resp1 = testRequest('GET', 'https://0.0.0.0:22432/handshake', headers, cookies, body)
except:
    raise Exception('Request failed with/without status code ')

peer_rsa_crt = sec3.decrypt(resp1['payload']['ciphertext'], resp1['payload']['iv'], sharedSecret)

/usr/lib/python3/dist-packages/urllib3/connectionpool.py:1015: InsecureRequestWarning: Unverified HTTPS request is being made to host '0.0.0.0'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


In [15]:
# TEST: Distribution Request
resp2 = {}
try:
    channelName = createId(5)
    channelName_hex_string = hex(int.from_bytes(channelName.encode(), 'big'))
    peer_hex_string = peer_rsa_crt.hex()
    encRes = sec3.encrypt(bytes.fromhex(channelName_hex_string[2:]), bytes.fromhex(peer_hex_string), sharedSecret)
    encIv = encRes['iv']
    encText = encRes['ciphertext']
    cookies = {
        'sessionId': resp1['sessionId'],
        'X-CSRFToken': resp1['csrfToken']
    }
    headers = {'Content-Type': 'application/json'}
    body = {'apiToken': apiTokenValue, 'userToken': userTokenValue, 'iv': encIv, 'channelName': encText}
    resp2 = testRequest('POST', 'https://0.0.0.0:22432/distribution', headers, cookies, body)
except Exception as e:
    print(e)
    raise Exception('Request failed with/without status code ')

/usr/lib/python3/dist-packages/urllib3/connectionpool.py:1015: InsecureRequestWarning: Unverified HTTPS request is being made to host '0.0.0.0'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


In [16]:
# TEST: API Ca Creation Request
resp3 = {}
try:
    cookies = {
        'sessionId': resp2['sessionId'],
        'X-CSRFToken': resp2['csrfToken']
    }
    headers = {'Content-Type': 'application/json'}
    body = {'apiToken': apiTokenValue, 'creatorToken': creatorTokenValue}
    resp3 = testRequest('POST', 'https://0.0.0.0:22432/cert/api/ca', headers, cookies, body)
except Exception as e:
    print(e)
    raise Exception('Request failed with/without status code ')

/usr/lib/python3/dist-packages/urllib3/connectionpool.py:1015: InsecureRequestWarning: Unverified HTTPS request is being made to host '0.0.0.0'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


In [17]:
# TEST: API Ca Request
resp4 = {}
try:
    cookies = {
        'sessionId': resp3['sessionId'],
        'X-CSRFToken': resp3['csrfToken']
    }
    headers = {'Content-Type': 'application/json'}
    body = {'apiToken': apiTokenValue, 'creatorToken': creatorTokenValue}
    resp4 = testRequest('GET', 'https://0.0.0.0:22432/cert/api/ca', headers, cookies, body)
except Exception as e:
    print(e)
    raise Exception('Request failed with/without status code ')

/usr/lib/python3/dist-packages/urllib3/connectionpool.py:1015: InsecureRequestWarning: Unverified HTTPS request is being made to host '0.0.0.0'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


In [18]:
# TEST: API Broker Crt Creation Request
resp6 = {}
try:
    cookies = {
        'sessionId': resp4['sessionId'],
        'X-CSRFToken': resp4['csrfToken']
    }
    headers = {'Content-Type': 'application/json'}
    body = {'apiToken': apiTokenValue, 'creatorToken': creatorTokenValue}
    resp6 = testRequest('POST', 'https://0.0.0.0:22432/cert/api/broker', headers, cookies, body)
except Exception as e:
    print(e)
    raise Exception('Request failed with/without status code ')

/usr/lib/python3/dist-packages/urllib3/connectionpool.py:1015: InsecureRequestWarning: Unverified HTTPS request is being made to host '0.0.0.0'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


In [19]:
# TEST: API Client Crt Creation Request
resp7 = {}
try:
    cookies = {
        'sessionId': resp6['sessionId'],
        'X-CSRFToken': resp6['csrfToken']
    }
    headers = {'Content-Type': 'application/json'}
    body = {'apiToken': apiTokenValue, 'creatorToken': creatorTokenValue}
    resp7 = testRequest('POST', 'https://0.0.0.0:22432/cert/api/client', headers, cookies, body)
except Exception as e:
    print(e)
    raise Exception('Request failed with/without status code ')

/usr/lib/python3/dist-packages/urllib3/connectionpool.py:1015: InsecureRequestWarning: Unverified HTTPS request is being made to host '0.0.0.0'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


In [20]:
# TEST: CAM Ca Creation Request
resp8 = {}
try:
    cookies = {
        'sessionId': resp7['sessionId'],
        'X-CSRFToken': resp7['csrfToken']
    }
    headers = {'Content-Type': 'application/json'}
    body = {'apiToken': apiTokenValue, 'creatorToken': creatorTokenValue}
    resp8 = testRequest('POST', 'https://0.0.0.0:22432/cert/cam/ca', headers, cookies, body)
except Exception as e:
    print(e)
    raise Exception('Request failed with/without status code ')

/usr/lib/python3/dist-packages/urllib3/connectionpool.py:1015: InsecureRequestWarning: Unverified HTTPS request is being made to host '0.0.0.0'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


In [21]:
# TEST: CAM Ca Request
resp9 = {}
try:
    cookies = {
        'sessionId': resp8['sessionId'],
        'X-CSRFToken': resp8['csrfToken']
    }
    headers = {'Content-Type': 'application/json'}
    body = {'apiToken': apiTokenValue, 'creatorToken': creatorTokenValue}
    resp9 = testRequest('GET', 'https://0.0.0.0:22432/cert/cam/ca', headers, cookies, body)
except Exception as e:
    print(e)
    raise Exception('Request failed with/without status code ')
print(resp9)
print(sec3.decrypt(resp9['payload']['caCrt']['ciphertext'], resp9['payload']['caCrt']['iv'], sharedSecret))
print(sec3.decrypt(resp9['payload']['caKey']['ciphertext'], resp9['payload']['caKey']['iv'], sharedSecret))

/usr/lib/python3/dist-packages/urllib3/connectionpool.py:1015: InsecureRequestWarning: Unverified HTTPS request is being made to host '0.0.0.0'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


{'payload': {'caCrt': {'iv': '74f563caf6b4151b25112c6534889d3d', 'ciphertext': 'f0bd4807efd5ffd0727880b1a2c77c51f6f4b3c6603c199f2fefa54d8916c9847b60c06c708c26cc1a04b7b7a49cee012f1dff2abd7dfa30073eacbdae75c20e80fed63550476bd23425953f8d69d0ad4c44a356fb29d3d60feb5e45e54f8f500752e5de8000a30ef3243d557aa718a8e76f697d59ced57147242462f9e1441a03fab7e90776e8846e7bfe29e109b60a6ee6abfd8b5d7bd285f5f77ad1c6ed4b66098d52f5726ae70a22290bbe0d9ec79e22b5f14f46d904ef6da7ed9d05ceb6e735453d92377166f5afa1ecbf5726b338e6b26c7b71ab22d7ef88ec0d386811b40b735bdbc94de5e0fba3c14350f80e32c4c09ee358c48477054feae17d1961e13c79c8def0e7c82679843181e6490075d9de52de5f60a4df9138c0999fb99be6fb0eb5e93b7268483d86c2fe6b237d1758652b79d31054e6c9e6ee95c2e5d73dcd285f433f03cc42a04a941c0545e4f5fcc45e9f750d5b8bb20df8f63fdc902c84464554a6db080cde33047467925583eb6cf76e34d798b4e99c6c11d02125854e0a008e41a73edc11003e7a08a93d727d7d68b4e45038cad797b4fa173e0ddac01c48ba36556599ded838daa48320fd00bed5df8799fb3f02e1ef3ba936470f0c42f37cd34c8bbd84f5cb

In [22]:
# TEST: CAM Broker Crt Creation Request
resp10 = {}
try:
    cookies = {
        'sessionId': resp9['sessionId'],
        'X-CSRFToken': resp9['csrfToken']
    }
    headers = {'Content-Type': 'application/json'}
    body = {'apiToken': apiTokenValue, 'creatorToken': creatorTokenValue}
    resp10 = testRequest('POST', 'https://0.0.0.0:22432/cert/cam/broker', headers, cookies, body)
except Exception as e:
    print(e)
    raise Exception('Request failed with/without status code ')

/usr/lib/python3/dist-packages/urllib3/connectionpool.py:1015: InsecureRequestWarning: Unverified HTTPS request is being made to host '0.0.0.0'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


In [23]:
# TEST: CAM Broker Crt Creation Request
resp11 = {}
try:
    cookies = {
        'sessionId': resp10['sessionId'],
        'X-CSRFToken': resp10['csrfToken']
    }
    headers = {'Content-Type': 'application/json'}
    body = {'apiToken': apiTokenValue, 'creatorToken': creatorTokenValue}
    resp11 = testRequest('POST', 'https://0.0.0.0:22432/als', headers, cookies, body)
except Exception as e:
    print(e)
    raise Exception('Request failed with/without status code ')

/usr/lib/python3/dist-packages/urllib3/connectionpool.py:1015: InsecureRequestWarning: Unverified HTTPS request is being made to host '0.0.0.0'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


In [24]:
resp12 = {}
try:
    cookies = {
        'sessionId': resp11['sessionId'],
        'X-CSRFToken': resp11['csrfToken']
    }
    headers = {'Content-Type': 'application/json'}
    body = {'apiToken': apiTokenValue, 'creatorToken': creatorTokenValue}
    resp12 = testRequest('GET', 'https://0.0.0.0:22432/als', headers, cookies, body)
except Exception as e:
    print(e)
    raise Exception('Request failed with/without status code ')

/usr/lib/python3/dist-packages/urllib3/connectionpool.py:1015: InsecureRequestWarning: Unverified HTTPS request is being made to host '0.0.0.0'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


In [25]:
# TEST: Deauth

In [26]:
print('RESULTS ECDHE TEST')
print('\nServer Response:')
print(resp0)
print('\nServer Response Validation:')
print(valEcdhe)
print('\n\nRESULTS RSA TEST')
print('\nServer Response:')
print(resp1)
print('\nPeer Cert:')
print(str(peer_rsa_crt))
print('\nPeer Cert Hex:')
print(str(peer_rsa_crt.hex()))
print('\n\nRESULTS DISTRIBUTION TEST')
print('\nServer Response:')
print(resp2)
print('\n\nRESULTS API CA CREATION TEST')
print('\nServer Response:')
print(resp3)
print('\n\nRESULTS API CA DISTRIBUTION TEST')
print('\nServer Response:')
print(resp4)
print(sec3.decrypt(resp4['payload']['caCrt']['ciphertext'], resp4['payload']['caCrt']['iv'], sharedSecret))
print(sec3.decrypt(resp4['payload']['caKey']['ciphertext'], resp4['payload']['caKey']['iv'], sharedSecret))
print('\n\nRESULTS API BROKER CREATION TEST')
print('\nServer Response:')
print(resp6)
print(sec3.decrypt(resp6['payload']['caCrt']['ciphertext'], resp6['payload']['caCrt']['iv'], sharedSecret))
print(sec3.decrypt(resp6['payload']['caKey']['ciphertext'], resp6['payload']['caKey']['iv'], sharedSecret))
print('\n\nRESULTS API CLIENT CREATION TEST')
print('\nServer Response:')
print(resp7)
print(sec3.decrypt(resp7['payload']['caCrt']['ciphertext'], resp7['payload']['caCrt']['iv'], sharedSecret))
print(sec3.decrypt(resp7['payload']['caKey']['ciphertext'], resp7['payload']['caKey']['iv'], sharedSecret))
print('\n\nRESULTS CAM CA CREATION TEST')
print('\nServer Response:')
print(resp8)
print('\n\nRESULTS CAM CA DISTRIBUTION TEST')
print('\nServer Response:')
print(resp9)
print(sec3.decrypt(resp9['payload']['caCrt']['ciphertext'], resp9['payload']['caCrt']['iv'], sharedSecret))
print(sec3.decrypt(resp9['payload']['caKey']['ciphertext'], resp9['payload']['caKey']['iv'], sharedSecret))
print('\n\nRESULTS API BROKER CREATION TEST')
print('\nServer Response:')
print(resp10)
print(sec3.decrypt(resp10['payload']['caCrt']['ciphertext'], resp10['payload']['caCrt']['iv'], sharedSecret))
print(sec3.decrypt(resp10['payload']['caKey']['ciphertext'], resp10['payload']['caKey']['iv'], sharedSecret))
print('\n\nRESULTS API CLIENT CREATION TEST')
print('\nServer Response:')
print(resp11)
print('\n\nRESULTS ALS KEY DISTRIBUTION TEST')
print('\nServer Response:')
print(resp12)


RESULTS ECDHE TEST

Server Response:
{'payload': {'digest': 'dc08237863be0de8c5ceb5a4fb087aac8b5ce7855dcaf1146ace90355a0fd09d', 'key': '2d2d2d2d2d424547494e205055424c4943204b45592d2d2d2d2d0a4d466b77457759484b6f5a497a6a3043415159494b6f5a497a6a304441516344516741452f73685232316a673533365236385437426d48784979746b743759510a6f4952567077536e724a57542b6173704446517342753363336f54374b6a6b592f636d666b617662665771785778644f315a4c6d4631385441413d3d0a2d2d2d2d2d454e44205055424c4943204b45592d2d2d2d2d0a'}, 'csrfToken': '167714db3274e38e3d8487e37bf7286dd37767c8IjEwYWZjMDY1MmMyODZlODIwYjE3ZWFlZjA3NTUwNWY0OGM4MzRjYWUi.ZFjyLA.hU61La2K4VadiF8nVriy0zF4jF0', 'sessionId': 'f20020be-91bd-4f8b-91ef-32e548373af7'}

Server Response Validation:
True


RESULTS RSA TEST

Server Response:
{'payload': {'iv': '71e46942c74c8ff10a99ba590a397f1e', 'ciphertext': '29d8bee45d82be7f5fa7e8eb5b149f1930d800e1e039d161b35cf7c3ee83333844652ef8ccbee359fdd4537f0983cd6db402365f15458964cb1263242148ae1ab73d9e345de3cb3fd9e1f236a863fc2bdc